In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

import nltk
from nltk.util import ngrams

In [23]:
def read_lyrics_data(file_name):
    return pd.read_csv(file_name)

# Model Builder

In [24]:
def get_vocab_words(genre_df, max_features, ngram_range):
    vocab_set = set()
    vectorizer = TfidfVectorizer(max_features = max_features, ngram_range = ngram_range, max_df = .999, min_df = .00001,\
                                stop_words = 'english')
    
    vectorizer.fit_transform(genre_df.Lyrics)
    genre_vocab = vectorizer.vocabulary_
    vocab_set.update(set(genre_vocab.keys()))
    return vocab_set

In [25]:
from sklearn.linear_model import LogisticRegression
def get_logistic_metrics(features, labels):
    penalty = ['l1', 'l2']
    regs = [2, 1, .1, .01]
    
    hyperparameters = dict(C=regs, penalty=penalty)
    model = LogisticRegression(random_state=0, solver = 'saga',class_weight='balanced', multi_class='multinomial', \
                               max_iter=3000)
    
    clf = GridSearchCV(model, hyperparameters, cv=2, verbose=0, n_jobs = 5)
    grid_model = clf.fit(features, labels)
    return grid_model

In [26]:
from sklearn.ensemble import RandomForestClassifier
def get_forest_metrics(features, labels):
    num_trees = [30, 40, 50]
    max_features = ['sqrt', 'log2']
    depth = [11, 13, 15]
    
    hyperparameters = dict(n_estimators=num_trees, max_depth=depth, max_features = max_features)
    model = RandomForestClassifier(min_samples_leaf = int(.05 * len(labels)), random_state=0, class_weight='balanced')
    
    clf = GridSearchCV(model, hyperparameters, cv=2, verbose=0, n_jobs = 5)
    grid_model = clf.fit(features, labels)
    return grid_model

In [27]:
def run_cross_validation(lyrics_df, ngram):
    unique_genres = lyrics_df.Genre.unique()
    
    features = lyrics_df.Lyrics
    labels = lyrics_df.Genre
    vocab_set = set()
    
    for genre in unique_genres: # Get top phrases for each genre
        genre_df = lyrics_df[lyrics_df.Genre == genre]
        vocab = get_vocab_words(genre_df, 300, (ngram, ngram))
        vocab_set.update(vocab)
    
    
    vectorizer = TfidfVectorizer(ngram_range = (ngram, ngram), vocabulary = vocab_set, stop_words = 'english')
    train_features = vectorizer.fit_transform(features)
    
    logistic_grid = get_logistic_metrics(train_features, labels)
   
    forest_grid = get_forest_metrics(train_features, labels)
   
    return logistic_grid, forest_grid

In [28]:
lyrics = read_lyrics_data("cleaned_lyrics.csv")
logistic_df = []
forest_df = []

for g in range(1, 7):
    run_cross_validation(lyrics, g)
    logistic_grid, forest_grid = run_cross_validation(lyrics, g)
    
    best_logistic = logistic_grid.best_params_
    best_logistic['gram'] = g
    best_logistic['score'] = logistic_grid.best_score_
    logistic_df.append(best_logistic)
    
    best_forest = forest_grid.best_params_
    best_forest['gram'] = g
    best_forest['score'] = forest_grid.best_score_
    forest_df.append(best_forest)

In [29]:
logistic_df = pd.DataFrame(logistic_df)
logistic_df.to_csv("logistic_metrics.csv", index = False)

forest_df = pd.DataFrame(forest_df)
forest_df.to_csv("forest_metrics.csv", index = False)

,Artist,Genre,Song,Lyrics
0,Megan Thee Stallion,Hip-Hop/R&B,Savage Remix,Queen'B want no smoke with me Okay Been turnt ...
1,Megan Thee Stallion,Hip-Hop/R&B,Cash Shit,Real hot girl shit Yeah I'm in my bag but I'm ...
2,Megan Thee Stallion,Hip-Hop/R&B,Savage,I'm that bitch Yeah Been that bitch still that...
3,Tyga,Hip-Hop/R&B,Taste,D.A Hah Slide on it pimp game with my pinky ri...
4,Tyga,Hip-Hop/R&B,Stimulated,Fuck what they talkin bout All a nigga hear is...
...,...,...,...,...
73,Logan Henderson,Pop,Sleepwalker,There wa a moment we said our goodbye I sat an...
74,Logan Henderson,Pop,Bite My Tongue,I ve never been the social type But I talk to ...
75,One Direction,Pop,Perfect,I might never be your knight in shining armor ...
76,One Direction,Pop,Drag Me Down,I ve got fire for a heart I'm not scared of th...
